In [ ]:
import requests
import orjson
from web3 import Web3
import asyncio
import multiprocessing
import aiohttp
import time
import re
from ens import ENS

In [ ]:
ETHERSCAN_API_URL = 'https://api.etherscan.io/api'
ETHERSCAN_API_KEY = 'HBPII4X833WAR5R3UUTIPKRC6VT6VNPZK3'

INFURA_ENDPOINT = 'https://mainnet.infura.io/v3/d39a3a75e0aa41f4a5334c561331ee83'
INFURE_WSS = 'wss://mainnet.infura.io/ws/v3/d39a3a75e0aa41f4a5334c561331ee83'
SECRET = '7b336d77672e46cf8f217d8f99a9d293'
PRJ_ID = 'd39a3a75e0aa41f4a5334c561331ee83'

w3 = Web3(Web3.HTTPProvider(INFURA_ENDPOINT))
ns =  ENS(Web3.HTTPProvider(INFURA_ENDPOINT))

In [ ]:
# get variables from etherscan, format it into dict and save
import json
exchange_addresses = {}
with open('exchange_addresses.txt') as f:
    lines = f.readlines()
for line in lines:
    parse = line.split('\t')
    exchange_addresses[parse[0]] = parse[1]



In [ ]:
# version 1, etherscan api and requests.

def get_addr_norm_txn_history(addr, url=ETHERSCAN_API_URL, key = ETHERSCAN_API_KEY):
    params = {
        'module':'account',
        'action':'txlist',
        'address': addr,
        'startblock':0,
        'endblock':99999999,
        'page':1,
        'offset':10000,
        'sort':'asc',
        'apikey':key
    }
    
    response = requests.get(url, params=params).json()['result']
    return response

def get_addr_internal_txn_history(addr, url=ETHERSCAN_API_URL, key = ETHERSCAN_API_KEY):
    params = {
        'module':'account',
        'action':'txlist',
        'address': addr,
        'startblock':0,
        'endblock':99999999,
        'page':1,
        'offset':10,
        'sort':'asc',
        'apikey':key
    }
    
    response = requests.get(url, params).json()['result']
    return response


def get_most_recent_block(url=ETHERSCAN_API_URL, key = ETHERSCAN_API_KEY):
    params = {
        'module':'proxy',
        'action':'eth_blockNumber',
        'apikey':key
    }
    response = requests.get(url, params).json()['result']
    return response

def get_block_transactions(block, url=ETHERSCAN_API_URL, key = ETHERSCAN_API_KEY):
    
    params = {  
        'module': 'proxy',  
        'action': 'eth_getBlockByNumber',  
        'tag': block, 
        'boolean': 'true',
        'apikey': key  
    }  
    response = requests.get(url, params).json()['result']
    return_dict ={
        'block':block,
        'transactions':response['transactions']
    }

    return return_dict

async def parse_transaction_for_creation(transaction_hash):
    
    return_dict = {}
    
    contractAddress = w3.eth.getTransactionReceipt(transaction_hash)['contractAddress']
    
    if contractAddress != None:
        
        return_dict = {
            'block':contractAddress['blockNumber'],
            'creator':contractAddress['from'],
        }    
    
        return return_dict

def is_transfer_from_ens_holder(from_addr, url=ETHERSCAN_API_URL, key = ETHERSCAN_API_KEY):
    
    try:
        domain = ns.name(from_addr)
        return True
    except ValueError as err:
        return False

# is_transfer_from_ens_holder('0x3cCeBA645566A30869651c14F567F26cC1a98EF8')



In [ ]:
method_ids = {
    '0x60806040': 'contractCreation',
    '0x60c06040': 'contractCreation',
    '0xfb3bdb41': 'swap',
    '0xfdacd576': 'setCompleted',
    '0x6ecd2306': 'mint',
    '0xa8a41c70': 'cancelOrder',
    '0xa22cb465': 'setApprovalForAll',
    '0x095ea7b3': 'approve',
    '0xab834bab': 'atomicMatch',
    '0x2e1a7d4d': 'withdraw',
    '0x6a761202': 'execTransaction',
    '0x1688f0b9': 'createProxyWithNonce',
    '0x0dcd7a6c': 'sendMultiSigToken',
    '0xd0e30db0': 'deposit',
    '0x1cff79cd': 'execute',
    '0xf14fcbc8': 'commit',
    '0x7263cfe2': 'addToAllowList',
    '0x7d533c1e': 'lockTokens',
    '0xf305d719': 'addLiquidityETH',
    '0x715018a6': 'renounceOwnership',
    '0xbc337182': 'setMaxTx',
    '0x5932ead1': 'setCooldownEnabled',
    '0xc9567bf9': 'openTrading',
    '0xff872602': 'removeStrictTxLimit',
    '0x76704de0': 'extendLockDuration',
    '0x5b0d5984': 'removeLiquidityETHWithPermitSupportingFeeOnTransferTokens',
    '0xa9059cbb': 'transfer',
    '0xac9650d8': 'multicall',
    '0xff897570': 'addToBlackList',
    '0x1bbae6e0': '_setMaxTxAmount',
    '0x5b0d5984': 'removeLiquidityETHWithPermitSupportingFeeOnTransferTokens',
    '0xa9059cbb': 'transfer',
    '0x1652e9fc': 'set_minter',
    '0x2195995c':'removeLiquidityWithPermit',
    '0x4b014e28':'setClaimState',
    '0x23b872dd': 'transferFrom',
    '0xa8a41c70': 'cancelOrder_',
    '0xfb3bdb41': 'swapETHForExactTokens',
    '0x8264fe98': 'buyPunk',
    '0x5f575529': 'metaMaskSwap',
    '0x9cec6392': 'rariblePurchase',
    '0x454a2ab3': 'bid',
    '0xf7a16963': 'registerWithConfig',
    '0x1896f70a': 'setResolver',
    '0x4946e206': 'rescueCat',
    '0xe1a6f014': 'offer',
    '0x40e58ee5': 'cancel',
    '0xbe040fb0': 'redeem',
    '0x7ff36ab5': 'swapExactEthForTokens',
    '0x18cbafe5': 'swapExactTokensForETH',
    '0xf2fde38b': 'transferOwnership',
    '0x52390c02': 'excludeFromRewards',
    '0x437823ec': 'excludeFromFee',
    '0xc0e6b46e': 'setRouterPercent',
    '0xd543dbeb': 'setMaxTxPercent',
    '0xc3c8cd80': 'manualSwap',
    '0x6fc3eaec': 'manualSend',
    '0xc0e6b46e': 'setRouterPercent',
    '0xded9382a': 'removeLiquidityETHWithPermit',
    '0x8af416f6': 'lockLPToken',
    '0x60491d24': 'relock',
    '0x2298207a': 'paraswapMainnet',
    '0x9e281a98': 'withdrawToken',
    '0x278b8c0e': 'cancelOrder',
    '0x0b66f3f5': 'multisendToken',
    '0x8f70ccf7': 'setTrading',
    '0x74010ece': 'setMaxTxnAmount',
    '0x98a5c315': 'setMinSwapTokensThreshold',
    '0xea1644d5': 'setMaxWalletSize',
    '0xed97c892': 'createNewAtomicSwapContract',
    '0x8f70ccf7': 'setTrading',
    '0xed97c892': 'createNewAtomicSwapContract',
    '0x2f9c4569': 'allowPreTrading',
    '0x00b8cf2a': 'blockBots',
    '0x6b999053': 'unblockBot',
    '0xa2bc66be': 'unstake'
}

with open('exch_addrs.txt') as f:
    exch_addrs = json.loads(f.read())

In [ ]:
block_nums = [13613462, 13618339,13618295,13615958,13454131,13618185, 13621662,
              13621618,13610187,13621586, 13621512,12534802,13619386,13484158,
              13610187, 13489862,13598928,13442242,13590273, 13488588] 

_h_txn = [hex(b) for b in block_nums]

def check_for_creation(hex_block):
    
    dataset = {
        'block':hex_block,
        'transactions': []
    }
    
    txns = [txn.hex() for txn in w3.eth.get_block(hex_block)['transactions']]
    
    for _t in txns:

        _r = w3.eth.get_transaction_receipt(_t)
        if _r['contractAddress'] != None:
    
            _d = {
                'from':_r['from'],
                'txn' :_r['transactionHash'].hex()
                
            }
            dataset['transactions'].append(_d)
            
    return dataset

def get_addr_norm_txn_history_str(block_contract_creation_arr):

    trx_arrs = {}
    for c in range(len(block_contract_creation_arr)):
        trx_arr = []
        f = block_contract_creation_arr[c]['from']
        # addr_txns = get_addr_norm_txn_history(f)
        # for t in addr_txns:
        #     # check transfer
        #     if len(t['input'])==2:
        #         if t['from'] in exch_addrs:
        #             trx_arr.append({'hash':t['hash'], 'method':'transfer','block': t['blockNumber'] ,'from_exch':True})
        #     else:
        #         _m = t['input'][:10]
        #         if _m in method_ids:
        #             trx_arr.append({'hash':t['hash'], 'method':method_ids[_m],'block': t['blockNumber']})
        #         else:
        #             trx_arr.append({'hash':t['hash'], 'method':_m,'block': t['blockNumber']})    
        
        # trx_arrs[f] = trx_arr

    return trx_arrs
        
def creator_source_funds_filter(creator_addr):

    creation_idx = None
    addr_txns = get_addr_norm_txn_history_str(creator_addr)
    for v in addr_txns.values():
        # find index of creation, in real-time this will be usually be 1 
        for i in range(len(v)):   
            if v[i]['method'] == 'contractCreation':
                creation_idx = i

    potential_transfers =v[:creation_idx]
    exchange_transfers = 0
    wallet_transfers = 0
    for i in potential_transfers:
        if 'from_exch' in i:
            exchange_transfers +=1
        else:
            wallet_transfers +=1

    if exchange_transfers >= wallet_transfers:
        return 1
    else:
        return 0

def prior_removal_of_liquidity_filter(creator_addr):    

    contract_creations = 0
    remove_liquidity = 0
    return_val = 0

    addr_txns = get_addr_norm_txn_history_str(creator_addr)

    # check txn count, (i initially had this in its own function but here prob better)

    if len(addr_txns) > 3: 
        return_val += 1
    else:
        return_val -= 0.25

    for v in list(addr_txns.values())[0]:
        if 'contractCreation' in v['method']:
            contract_creations += 1
        if 'removeLiquidity' in v['method']:
            remove_liquidity += 1

    return_val -= remove_liquidity
    if contract_creations > 1:
        return_val -= contract_creations*0.25
    else:
        return return_val

    # blocks_since
    
def backwards_filters(addr):
    
    total = 0
    total += creator_source_funds_filter(addr)
    total += prior_removal_of_liquidity_filter(addr)

    return total

backwards_filters(['0x8A5A3E16f9CC221fd827246F3E57212a7A222fFB'])

In [ ]:
from multiprocessing import Pool

current_block = get_most_recent_block()
async def process_block(focus_block, url=ETHERSCAN_API_URL, key = ETHERSCAN_API_KEY):

    block_obj = get_block_transactions(focus_block)
    hashes = [txn['hash']for txn in block_obj['transactions']]
    
    await asyncio.gather(*[parse_transaction_for_creation(h) for h in hashes])
    # p = Pool(5)
    # r = p.map(parse_transaction_for_creation, hashes)

        
await process_block(current_block)

In [ ]:
result_objs = []

with Pool(processes=8) as pool:
    for _ in range(n):
        result = pool.apply_async(f, (n,))
        result_objs.append(result)
    
    results = [result.get() for result in result_objs]
    print(len(results), np.mean(results), np.var(results))

In [ ]:
from multiprocessing import JoinableQueue
from multiprocessing.context import Process

class Renderer:
    queue = None

    def __init__(self, nb_workers=4):
        self.queue = JoinableQueue()
        self.processes = [Process(target=self.upload) for i in range(nb_workers)]
        for p in self.processes:
            p.start()

    def render(self, item):
        self.queue.put(item)

    def upload(self):
        while True:
            item = self.queue.get()
            if item is None:
                break

            # process your item here

            self.queue.task_done()

    def terminate(self):
        """ wait until queue is empty and terminate processes """
        self.queue.join()
        for p in self.processes:
            p.terminate()

r = Renderer()
r.render(item1)
r.render(item2)
r.terminate()

In [ ]:
from blocknative.stream import Stream
import json

stream = Stream('47845801-21ab-46e6-b7d9-ecce8ae3be5b')